<a href="https://colab.research.google.com/github/Gregrs400/cmpsc472Project2/blob/set-fire/cmpsc472Project2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install git

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.11).
0 upgraded, 0 newly installed, 0 to remove and 49 not upgraded.


In [ ]:
!git clone https://github.com/Gregrs400/cmpsc472Project2.git


Cloning into 'cmpsc472Project2'...
remote: Enumerating objects: 42, done.
remote: Counting objects: 100% (42/42), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 42 (delta 11), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (42/42), 17.31 KiB | 8.66 MiB/s, done.
Resolving deltas: 100% (11/11), done.


In [ ]:
%%bash
cd
ls
if [ -d  "cmpsc472Project2" ]; then
cd cmpsc472Project2
git pull
else
git clone https://github.com/Gregrs400/cmpsc472Project2.git;
cd cmpsc472Project2
fi
ls

cmpsc472Project2
Already up to date.
cmpsc472Project2
cmpsc472Project2.c
cmpsc472Project2.ipynb
Ideas.md


In [214]:
%%writefile cmpsc472Project2.c
#include <stdio.h>
#include <stdlib.h>
#include <string.h>
#include <unistd.h>
#include <sys/wait.h>
#include <sys/time.h>
#include <sys/resource.h>
#include <time.h>
#include <stdbool.h>

#define MAX_COLUMNS 49

/*
TODO:
- add global pool of resources to fight fires. This should be an int with an arbritrary value, e.g. 30 resources
- add fire fighting resources throughout the landscape
- determine whether to have multiple processes or multiple threads deal with the wildfires - Note: have parent process act as dispatch, have a child process search for wildfires, have
multiple threads from that child process fight the wildfires
- implement inter-process communication
- use semaphores/mutexes to lock critical sections
- have logic for extinguishing wildfires, add resources back to global resource pool when a fire is extinguished
*/

int global_resources = 50;

struct Landscape {
    int rows;
    int columns;
    int (*grid)[MAX_COLUMNS];
};

void printLandscape(struct Landscape *landscape);

struct Fire {
  int row;
  int col;
  int priority;
  int requiredResources;
};


// Function to set a random position in the array to 1, which represents a wildfire.
struct Fire setFire(struct Landscape *landscape, int min, int max) {
  struct Fire fire;
  // Seed the random number generator
  srand(time(NULL));
  // Randomly select a row
  fire.row = rand() % (max - min + 1) + min;
  // Randomly select a column
  fire.col = rand() % (max - min + 1) + min;
  // Set the value at the selected row and column to 1
  landscape -> grid[fire.row][fire.col] = 1;
  printf("array value at position %d,%d: %d\n", fire.row, fire.col, landscape -> grid[fire.row][fire.col]);
  // Randomly assign a priority. Lower priorities have highest priority.
  fire.priority = rand() % (max - min + 1) + min;
  // Randomly assign a required number of resources to extinguish the fire.
  fire.requiredResources = rand() % (max - min + 1) + min;
  printf("Fire has priority %d and requires %d resources.\n", fire.priority, fire.requiredResources);
  return fire;
}

// Prints out the entire landscape array.
void printLandscape(struct Landscape *landscape) {
  for (int i = 0; i < landscape->rows; i++) {
    for (int j = 0; j < landscape->columns; j++) {
      if (landscape -> grid[i][j] == 1)
      {
        printf("\033[1;31m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 0)
      {
        printf("\033[1;32m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      else if (landscape -> grid[i][j] == 5)
      {
        printf("\033[1;34m%d\033[0m ", landscape->grid[i][j]);
        continue;
      }
      printf("%d ", landscape->grid[i][j]);
    }
    // Print a newline at the end of each row.
    printf("\n");
  }
}


int main() {
  // Create pipes for IPC
  int pipe_fd[2];
  if (pipe(pipe_fd) == -1) {
    perror("pipe");
    exit(EXIT_FAILURE);
  }

  struct Landscape *landscape = (struct Landscape *)malloc(sizeof(struct Landscape));

  if (landscape == NULL) {
      fprintf(stderr, "Memory allocation failed\n");
      return 1;
  }

  // Define number of rows
  landscape -> rows = 49;
  // Define number of columns
  landscape -> columns = 49;
  // Create array to represent the landscape
  int landscapeGrid[landscape -> rows][landscape -> columns];
  // setting the array to the landscape struct
  landscape -> grid = landscapeGrid;
  int totalGridRows = landscape -> rows;
  int totalGridColumns = landscape -> columns;
  // Fill array with 0s
  for (int i = 0; i < totalGridRows; i++)
  {
      for (int j = 0; j < totalGridColumns; j++)
      {
        if (i == totalGridRows / 2)
        {
            landscape -> grid[i][j] = 5;
            if (j == totalGridColumns / 2)
            {
              landscape -> grid[i][j] = 5;
              continue;
            }
        }
        landscape -> grid[i][j] = 0;
      }
  }

  // Number of fires to be set.
  int numOfFires = 5;
  // Amount of time to wait between setting fires.
  int waitTime;
  // Create array of Fire structs to store info on each fire set.
  struct Fire fires[numOfFires];
  // For loop to set the fires.
  for (int j = 0; j < numOfFires; j++) {
    // Stores attributes (row, col, priority, number of required resources) about the fire in jth element of Fire struct array.
    fires[j] = setFire(landscape, 0, 19);
    printLandscape(landscape);
    printf("\n");

    // Seed random number generator
    srand(time(NULL));
    // Wait between 1 and 10 seconds before setting another fire.
    waitTime = rand() % (10 - 1 + 1) + 1;
    // Wait a random amount of time before setting another fire.
    sleep(waitTime);
  }


/*
Note: How to use a child and parent process. Have the child process set a boolean flag value equal to true (1) after it completes searching through the landscape. Then have the child process sleep.
While the child is searching, it should communicate the coordinates, priority, and number of resources to the parent process. While the child process is sleeping, call wait().
Maybe have the parent process execute a while loop. While completeSearch = false, use continue statement, unless a fire is being communicated via IPC. Once the boolean flag is true, exit the loop, call wait()
*/


  // Now that all fires are set, create a child process to search for fires
  pid_t child_pid = fork();
  if (child_pid == 0) {
    // Counter to iterate through fires array
    int k = 0;
    // Search for fires in the landscape.
    // If one is found, use IPC to communicate that with the parent process.
    // Close read end of pipe
    close(pipe_fd[0]);
    // Search landscape for fires
    for (int i = 0; i < landscape -> rows; i++) {
      for (int j = 0; j < landscape ->columns; j++) {
        if (landscape -> grid[i][j] == 1) {
          // Communicate these coordinates to parent process
          write(pipe_fd[1], &i, sizeof(i));
          write(pipe_fd[1], &j, sizeof(j));

          // Communicate priority to parent process
          //write(pipe_fd[1], &fires[k].priority, sizeof(fires[k].priority));

          // Communicate number of resources to parent process
          write(pipe_fd[1], &fires[k].requiredResources, sizeof(fires[k].requiredResources));
          // Increment counter to get required resources for next fire
          k++;
        }
      }
    }
    // After child finishes search of landscape, notify the parent
    int complete = -1;
    write(pipe_fd[1], &complete, sizeof(-1));
    write(pipe_fd[1], &complete, sizeof(-1));

    // Close write end of pipe
    close(pipe_fd[1]);

  } else if (child_pid > 0) {
    // Parent process
    // Variables to store the row, column, requiredResources passed from child process
    int r, c, rr;
    // Close write end of pipe
    close(pipe_fd[1]);

    // While child process searches for fires, listen for IPC from child process
    while (true) {
      read(pipe_fd[0], &r, sizeof(r));
      if (r != -1)
      {
        printf("Fire row: %d\n", r);
      }
      read(pipe_fd[0], &c, sizeof(c));
      if (c != -1)
      {
        printf("Fire col: %d\n", c);
        read(pipe_fd[0], &rr, sizeof(rr));
        printf("Num of resources: %d\n", rr);
      }

      printf("\n");

      if (r == -1 && c == -1) {
        // Child process has finished searching for fires
        break;
      }
    }
    // Wait to ensure child process has finished executing.
    wait(NULL);

  } else {
    perror("Failed to create child process.");
    exit(EXIT_FAILURE);
  }

  return 0;
}

Overwriting cmpsc472Project2.c


In [215]:
%%shell
gcc cmpsc472Project2.c -o cmpsc472Project2
./cmpsc472Project2

array value at position 4,18: 1
Fire has priority 8 and requires 4 resources.
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 
0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0